<a href="https://colab.research.google.com/github/NancySunnn/BERTopic_covid_tweet/blob/main/tweet_covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python3 --version

Python 3.8.16


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import re
from string import punctuation

In [ ]:
!pip3 install bertopic

In [4]:
import os
import bertopic

In [5]:
df = pd.read_csv('Corona_NLP_train.csv', encoding ='latin1')

In [6]:
df.head(10)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,As news of the regionÂs first confirmed COVID...,Positive
6,3805,48757,"35.926541,-78.753267",16-03-2020,Cashier at grocery store was sharing his insig...,Positive
7,3806,48758,Austria,16-03-2020,Was at the supermarket today. Didn't buy toile...,Neutral
8,3807,48759,"Atlanta, GA USA",16-03-2020,Due to COVID-19 our retail store and classroom...,Positive
9,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,"For corona prevention,we should stop to buy th...",Negative


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       41157 non-null  int64 
 1   ScreenName     41157 non-null  int64 
 2   Location       32567 non-null  object
 3   TweetAt        41157 non-null  object
 4   OriginalTweet  41157 non-null  object
 5   Sentiment      41157 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.9+ MB


In [118]:
#text processing

def clean(text):
    #remove @ppl, url
    output = re.sub(r'https://\S*','', text)
    output = re.sub(r'@\S*','',output)
    
    #remove \r, \n
    rep = r'|'.join((r'\r',r'\n'))
    output = re.sub(rep,'',output)

    rep_covid = r'|'.join((r'\W*covid\S*',r'\W*COVID\S*',r'\W*coronavirus\S*', r'\W*Covid\S*',r'\W*Coronavirus\S*'))
    output = re.sub(rep_covid,'', output)

    #remove duplicated punctuation
    output = re.sub(r'([!()\-{};:,<>./?@#$%\^&*_~]){2,}', lambda x: x.group()[0], output)
    
    #remove extra space
    output = re.sub(r'\s+', ' ', output).strip()
    
    #remove string if string only contains punctuation
    if sum([i.isalpha() for i in output])== 0:
        output = ''
        
    #remove string if length<5
    if len(output.split()) < 5:
        output = ''

    return output

In [119]:
df['tweet_clean0'] = df['OriginalTweet'].apply(clean)

In [120]:
[i for i in df['tweet_clean0'][:20]]

['',
 'advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order',
 'Australia: Woolworths to give elderly, disabled dedicated shopping hours amid outbreak',
 "My food stock is not the only one which is empty.PLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. Stay calm, stay safe #confinement #Confinementotal #ConfinementGeneral",
 "Me, ready to go at supermarket during the outbreak.Not because I'm paranoid, but because my food stock is litteraly empty. The is a serious thing, but please, don't panic. It causes shortage #restezchezvous #StayAtHome #confinement",
 'As news of the regionÂ\x92s first confirmed case came out of Sullivan County last week, people flocked to area stores to purchase cleaning supplies, hand sanitizer, food, toilet paper and other goods, 

In [121]:
df = df.drop_duplicates()
df2 = df[df['tweet_clean0'] != '']
sum(df['tweet_clean0']=='')

282

In [122]:
#length
print(len(max(df2['tweet_clean0'], key=len)))
print(max(df2['tweet_clean0'], key=len))

300
MDOC Horhn MS65 50 MS brown MISSISSIPPI PRISONERS ARE GOING ON OFFICIAL FOOD amp WATER STRIKE APRIL 9 2020 TO PROTEST THE 19 PANDEMIC amp LACK OF PREVENTENTIVE MEASURES TAKEN BY MDOC STAFF SHORTAGE INHUMANE LIVING CONDITIONS POOR FOOD QUALITY QUALITY ABUSE amp DEMAND MSLEG FINALIZE amp ENACT SB 2123


In [13]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

In [14]:
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [123]:
#prepare embedding first
doc = df2['tweet_clean0'].tolist()
embedding = sentence_model.encode(doc)

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
from hdbscan import HDBSCAN

In [91]:
nlp = spacy.load('en_core_web_sm')
vectorizer_model = CountVectorizer(ngram_range=(1,2), stop_words=nlp.Defaults.stop_words)

In [162]:

cluster_model = HDBSCAN(min_cluster_size=300, min_samples=5, metric='euclidean', prediction_data = True)

topic_model = BERTopic(embedding_model=sentence_model, language='English',verbose=True,
                      calculate_probabilities=False, vectorizer_model = vectorizer_model, n_gram_range=(1,2), min_topic_size = 100, hdbscan_model=cluster_model, diversity=0.3)
topic, probs = topic_model.fit_transform(doc, embedding)


2023-01-05 22:41:12,435 - BERTopic - Reduced dimensionality
2023-01-05 22:41:14,563 - BERTopic - Clustered reduced embeddings


In [163]:
freq = topic_model.get_topic_freq()
print(freq)
print(len(freq))

    Topic  Count
0      -1  16922
1       0   3681
2       1   2343
3       2   2324
4       3   2154
5       4   1570
6       5   1439
7       6   1203
8       7   1066
9       8   1010
10      9    963
11     10    828
12     11    710
13     12    689
14     13    624
15     14    549
16     15    444
17     16    437
18     17    429
19     18    417
20     19    371
21     20    354
22     21    348
23


In [164]:
topic_model.get_topic(0)

[('sanitizer', 0.05514049036615613),
 ('hand', 0.047870927311877656),
 ('hand sanitizer', 0.04143626260712703),
 ('gloves', 0.018025320643466916),
 ('sanitizers', 0.011133905829889741),
 ('grocery store', 0.009137088157439119),
 ('handsanitizer', 0.009040215126408771),
 ('face masks', 0.00777653245207144),
 ('wash hands', 0.007552815822544595),
 ('hand sanitizers', 0.006063581394332705)]

In [160]:
#topic_model.visualize_hierarchy()
topic_model.visualize_heatmap()

In [165]:
topic_model.visualize_barchart()

In [166]:
topic_model.visualize_topics()

In [172]:
# Reduce outliers
new_topics = topic_model.reduce_outliers(doc, topic)b

100%|██████████| 17/17 [00:05<00:00,  2.97it/s]


In [ ]:
new_topics

In [174]:
from collections import Counter
new_topic_dict = dict(Counter(new_topics))

In [175]:
sorted(new_topic_dict, key = new_topic_dict.get, reverse=True)[:5]

[4, 0, 2, 7, 1]

In [177]:
df2['topic'] = new_topics

In [181]:
df2['topic'].value_counts()

 4     4885
 0     3921
 2     3264
 7     2881
 1     2817
 5     2674
 6     2624
 3     2398
 9     2019
 10    1989
 13    1664
 8     1537
 14    1272
 18    1148
 19    1022
 12     900
 11     839
 16     815
 15     625
 17     570
 20     543
 21     464
-1        4
Name: topic, dtype: int64

In [197]:
#guide topic modeling
seed_topic_list = [['vaccine','drug','testing','fda'],
                   ['buying','shortage','panic','stock'],
                   ['mask','socialdistancing','wash','sanitizer']]
cluster_model = HDBSCAN(min_cluster_size=300, min_samples=5, metric='euclidean', prediction_data = True)

guide = BERTopic(seed_topic_list = seed_topic_list, embedding_model=sentence_model, language='English',verbose=True,
                      calculate_probabilities=False, vectorizer_model = vectorizer_model, n_gram_range=(1,2), min_topic_size = 100, hdbscan_model=cluster_model, diversity=0.3)
g_topic, probs = guide.fit_transform(doc)

Batches:   0%|          | 0/1278 [00:00<?, ?it/s]

2023-01-06 00:41:30,145 - BERTopic - Transformed documents to Embeddings


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-01-06 00:42:34,226 - BERTopic - Reduced dimensionality
2023-01-06 00:42:38,341 - BERTopic - Clustered reduced embeddings


In [198]:
guide.get_topic_freq()

,Topic,Count
0,-1,14345
1,0,4106
2,1,3963
3,2,3270
4,3,2341
5,4,2126
6,5,1809
7,6,1470
8,7,1431
9,8,775


In [204]:
guide.visualize_heatmap()

In [203]:
guide.visualize_barchart()

In [202]:
guide.visualize_hierarchy()

In [201]:
guide.get_representative_docs()

['Direct-to-consumer, eCommerce behavior is being accelerated. Is it here to stay? #ecommerce #instoreexperience',
 'Thu Pham, our Brand Performance Consultant has gathered in-depth data insights from key markets to show the impact of on consumer behaviour. Get more insights here:',
 "In light of outbreaks, consumers are increasingly turning to ecommerce, with 1 in 5 saying they're doing more online shopping as a result. Discover the impact of on ecommerce by checking out statistics from our latest research study."]

In [207]:
keywords_list = list()
for i in range(0, 19):
  k = [x[0] for x in guide.get_topic(i)]
  keywords_list.append(k)
keywords_list

[['shopping',
  'online shopping',
  'pandemic',
  'consumers',
  'consumer behavior',
  'shopping online',
  'grocery',
  'ecommerce',
  'retail',
  'trends'],
 ['sanitizer',
  'hand sanitizer',
  'gloves',
  'sanitizers',
  'soap',
  'grocery store',
  'handsanitizer',
  'face masks',
  'wash hands',
  'safe'],
 ['panic',
  'panic buying',
  'food bank',
  'supermarket',
  'crisis',
  'meat',
  'pandemic',
  'farmers',
  'demand food',
  'donations'],
 ['grocery',
  'supermarket',
  'employees',
  'store workers',
  'nurses',
  'supermarket workers',
  'store employees',
  'delivery drivers',
  'supermarket staff',
  'clerks'],
 ['toiletpaper',
  'toilet paper',
  'toiletpapercrisis',
  'quarantine',
  'toiletpaperapocalypse',
  'supermarket',
  'toilet roll',
  'toiletpaper toiletpapercrisis',
  'toiletpaper toiletpaperpanic',
  '2020'],
 ['oil',
  'oil prices',
  'gas prices',
  'low',
  'saudi',
  'opec',
  'demand',
  'economy',
  'price war',
  'crude oil'],
 ['socialdistancing'

'/content'

In [209]:
guide.save('/content/drive/MyDrive/Colab Notebooks/bertopic/tweet_model_guide')

In [178]:
df2.to_csv('tweet_covid_result.csv', index=False)

In [210]:
#save model 
topic_model.save('/content/drive/MyDrive/Colab Notebooks/bertopic/tweet_model')